In [6]:
!pip install transformers
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [16]:
from transformers import pipeline

# 1. Setup classifier
classifier = pipeline("text-classification", model="roberta-base-openai-detector", device=-1)

# 2. Setup text generation (humanizing) pipeline
# We'll use a T5 model to paraphrase the text
paraphraser = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws", device=-1)

def detect_human_or_machine(text):
    result = classifier(text)[0]
    label = result['label']  # "REAL" means Human, "FAKE" means Machine
    confidence = result['score']
    if label == "FAKE":
        return "Machine"
    else:
        return "Human"

def humanize_text(text):
    prompt = f"paraphrase: {text} </s>"
    paraphrased = paraphraser(prompt, max_length=256, num_return_sequences=1, do_sample=True)[0]['generated_text']
    return paraphrased

# ---- Main Program ----
user_text = input("Enter your text: ")

label = detect_human_or_machine(user_text)
print(f"\nDetected: {label}")

if label.lower() == "machine":
    print("\nConverting to more human-like text...")
    new_text = humanize_text(user_text)
    print("\nHere is the humanized text:")
    print(new_text)
else:
    print("\nYour text is already human-like! 🎯")


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu


Enter your text: Would you also like me to give you a slightly nicer UI (like a text box + submit button inside Colab) instead of command-line typing?

Detected: Human

Your text is already human-like! 🎯


In [17]:
!pip install transformers
!pip install torch
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.7 MB/s eta 0:00:00


In [21]:
import ipywidgets as widgets
from transformers import pipeline

# Load Hugging Face models only once
print("Loading models... Please wait...")
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)
paraphraser = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws", device=-1)
print("Models loaded successfully! ✅\n")

# Functions
def detect_human_or_machine(text):
    candidate_labels = ['Human', 'Machine']
    result = classifier(text, candidate_labels)
    label = result['labels'][0]  # Best prediction
    return label

def humanize_text(text):
    prompt = f"paraphrase: {text} </s>"
    result = paraphraser(prompt, max_length=256, num_return_sequences=1, do_sample=True)
    humanized_text = result[0]['generated_text']
    return humanized_text

# UI function to process the input
def on_button_click(b):
    user_text = text_box.value.strip()  # Get text from text box

    if user_text.lower() == "exit":
        print("Exiting program. Goodbye! 👋")
        return

    # Detect
    label = detect_human_or_machine(user_text)
    output_label.value = f"\nDetected: {label}"

    # Humanize if machine-written
    if label.lower() == "machine":
        output_text.value = "Converting to more human-like text...\n" + humanize_text(user_text)
    else:
        output_text.value = "Your text is already human-like! 🎯"

# Text input widget (textbox)
text_box = widgets.Textarea(
    value='',
    placeholder='Enter your text here...',
    description='Text:',
    layout=widgets.Layout(width='100%', height='150px')
)

# Output text for results
output_label = widgets.Label(value="Detection result will appear here...")
output_text = widgets.Textarea(
    value='',
    placeholder='Converted text will appear here...',
    description='Result:',
    layout=widgets.Layout(width='100%', height='200px'),
    disabled=True
)

# Button to trigger processing
button = widgets.Button(description="Process Text")
button.on_click(on_button_click)

# Display UI
display(text_box, button, output_label, output_text)


Loading models... Please wait...


Device set to use cpu
Device set to use cpu


Models loaded successfully! ✅



Textarea(value='', description='Text:', layout=Layout(height='150px', width='100%'), placeholder='Enter your t…

Button(description='Process Text', style=ButtonStyle())

Label(value='Detection result will appear here...')

Textarea(value='', description='Result:', disabled=True, layout=Layout(height='200px', width='100%'), placehol…

In [22]:
!pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 92.7 MB/s eta 0:00:00


In [23]:
import ipywidgets as widgets
from transformers import pipeline
from spellchecker import SpellChecker

# Load Hugging Face models only once
print("Loading models... Please wait...")
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)
paraphraser = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws", device=-1)
print("Models loaded successfully! ✅\n")

# Initialize spell checker
spell = SpellChecker()

# Function for spell checking
def correct_spelling(text):
    words = text.split()
    corrected_text = []
    for word in words:
        corrected_word = spell.correction(word)
        corrected_text.append(corrected_word)
    return " ".join(corrected_text)

# Functions
def detect_human_or_machine(text):
    candidate_labels = ['Human', 'Machine']
    result = classifier(text, candidate_labels)
    label = result['labels'][0]  # Best prediction
    return label

def humanize_text(text):
    prompt = f"paraphrase: {text} </s>"
    result = paraphraser(prompt, max_length=256, num_return_sequences=1, do_sample=True)
    humanized_text = result[0]['generated_text']
    return humanized_text

# UI function to process the input
def on_button_click(b):
    user_text = text_box.value.strip()  # Get text from text box

    if user_text.lower() == "exit":
        print("Exiting program. Goodbye! 👋")
        return

    # Spell check the text
    corrected_text = correct_spelling(user_text)
    output_text.value = f"Corrected text: {corrected_text}\n"

    # Detect
    label = detect_human_or_machine(corrected_text)
    output_label.value = f"\nDetected: {label}"

    # Humanize if machine-written
    if label.lower() == "machine":
        output_text.value += "\nConverting to more human-like text...\n" + humanize_text(corrected_text)
    else:
        output_text.value += "\nYour text is already human-like! 🎯"

# Text input widget (textbox)
text_box = widgets.Textarea(
    value='',
    placeholder='Enter your text here...',
    description='Text:',
    layout=widgets.Layout(width='100%', height='150px')
)

# Output text for results
output_label = widgets.Label(value="Detection result will appear here...")
output_text = widgets.Textarea(
    value='',
    placeholder='Converted text will appear here...',
    description='Result:',
    layout=widgets.Layout(width='100%', height='200px'),
    disabled=True
)

# Button to trigger processing
button = widgets.Button(description="Process Text")
button.on_click(on_button_click)

# Display UI
display(text_box, button, output_label, output_text)


Loading models... Please wait...


Device set to use cpu
Device set to use cpu


Models loaded successfully! ✅



Textarea(value='', description='Text:', layout=Layout(height='150px', width='100%'), placeholder='Enter your t…

Button(description='Process Text', style=ButtonStyle())

Label(value='Detection result will appear here...')

Textarea(value='', description='Result:', disabled=True, layout=Layout(height='200px', width='100%'), placehol…